# Generate initial training set from SHARC-outputs

This script shows how to obtain an initial training set starting from calculations carried out with SHARC using Wigner sampling.

In [1]:
import ase.io
import ase
import numpy as np
import schnetpack as spk
from ase.units import Bohr
import os
SHARC="/user/julia/software/SchNarc/sharc/source/../bin/"
#import sharc
import schnarc
#print(help(schnarc))
from schnarc.utils import read_QMout

## Read the QM outputs and generate the training set


Note that units are all given in a.u. and that the geometries need to be saved in a.u. as well

In [3]:
#iterating over all files
nfiles = 100
#define number of states
ntriplets = 0
nsinglets = 2
nstates = nsinglets + 3 * ntriplets

#define number of atoms
natoms = 12


#read geometries --> these are in Angström!
geoms = ase.io.read("./InitialConditions/initconds.xyz",":")

#transform geoms into a.u.
atoms=[]
for i in range(len(geoms)):
    atoms.append(ase.atoms.Atoms(geoms[i].get_atomic_numbers(),geoms[i].get_positions()/Bohr))


#for conversion of atoms into bohr
from ase.units import Bohr

#data dictionary for updating the data base
data = {}

path="./InitialConditions/ICOND"
#we don't have spin-orbit couplings
socs=False

#read properties
for ifile in range(nfiles):
    filename = "%s_%05d/QM.out"%(path,ifile)
    data[ifile]={}
    #the last number is the threshold for overlaps. Do not change this value unless you know what it means and are clear about the consequences.
    data[ifile]=read_QMout(filename,natoms,socs,nsinglets,ntriplets,0.5)




## Including phase correction

If you wish to include phase correction, you need to have computed the overlap matrix between your chosen starting geometry and every other geometry you want to include in the training set.
If you don't want to do phase correction, skip this section and go directly to the section "Making the data set".

### QM calculations: getting the phase vectors

In case you have carried out overlap calculations, you will notice that there is an additional entry called "phases".
If the phases are not realiable enough, this entry will be missing. You can iterate over the data to find out which data points require additional interpolation

In [4]:
phasesnotfound = []
for i in range(len(data)):
    if "phases" in data[i]:
        pass
    else:
        #print("Folder ", i," requires interpolation to get phases.")
        phasesnotfound.append(i)
print(len(phasesnotfound), "data points do not contain reliable phases.")

98 data points do not contain reliable phases.


### Interpolation between geometries with insufficiently large overlaps

As you can see, only a few calculations have sufficiently large overlaps with the chosen starting geometry. All these calculations require interpolation between the starting geometry and the desired geometry. An overlap calculation has to be carried out from the starting geometry to the geometry that should be included in the set.
The phases for each calculation have to be found and multiplied with the phases of the previous geometry.

You can find an example in the folder: Interpolation_Geometry1
* start.xyz: geometry of the chosen reference point
* end.xyz: geometry which should be included in the training set

You can also give a number of interpolation steps, this is the last argument you add. In this case we have chosen 30 geometries interpolations.

In [4]:
# we now execute the interpolation script

from schnarc.utils import interpolate
#from interpolation import interpolate
start = "start.xyz"
end = "end.xyz"
interpolate(start,end,natoms,nsinglets,ntriplets,30)

#### Interpolate:
After interpolation (script is provided in the schnarc-utils), you will receive several files named "interpolateXX.xyz". Each file is one step in the interpolation. all geometries will be saved as separate files named interpolateXX.xyz with XX being the index of ther interpolation step. Geometries can be visualized with e.g. molden. Remember the units are in Bohr.

In this tutorial, all geometries are saved already in the folder "Interpolation_Geometry1". Each folder Calc_XXXXX corresponds to a folder of the interpolated structure. 
You can then carry out each calculation serially and compute the wave function overlaps. If you wish to include these data points in the training set, then compute all desired properties for each step. Note that the calculations cannot be done in parallel and need to be done after one another.

You will already be provided with the folders including the computations carried out.

We will now read in all properties and the phasevectors and skip the actual QM calculations.

In [5]:
#iterating over all files
nfiles = 30

#read geometries
geoms = ase.io.read("Interpolation_Geometry1/geometries.xyz",":")

#data dictionary for updating the data base
datainterpolate = {}
#TODO
for i in range(nfiles):
    filename = "Interpolation_Geometry1/Calc_%05d/QM.out"%i
    
    datainterpolate[i]=read_QMout(filename,natoms,socs,nsinglets,ntriplets,0.5)

In [6]:
phases_found = True 
for i in range(len(datainterpolate)):
    if "phases" not in datainterpolate[i]:
        print("Phase correction was not successful for interpolation step", i)
        phases_found= False
if phases_found == True:
    print("All data points could be corrected and data can be phase corrected.")

All data points could be corrected and data can be phase corrected.


Now we need to multiply each phase vector to find the phase vector of the final geometry

In [7]:
final_phase = np.ones(nsinglets+ntriplets)
for i in range(len(datainterpolate)):
    final_phase = final_phase * datainterpolate[i]["phases"]
    datainterpolate[i]["phases"] = final_phase


As you can see, the interpolation and search of the phase vector was successfull and we can now proceed to correct our data. You can find more details about the phase correction algorithm in Ref. 1. 
For all other calculations that required interpolation, we will for now assume all phasevectors have been found and are (1,1).

## Phase correcting data

In this subsection, we will now use the found phasevectors to correct our data.
We will assume we have found all phasevectors (they are set to +1 for every calculation where we could not found phases for now). 

### Correct interpolated geometries

Note that for interpolated geometries we need to consecutively multiply the phase vector with every previous phase vector. We have already done this for the interpolated data points.

In [8]:
#load the function to correct phases from SchNarc

from schnarc.utils import correct_phases

# we give the uncorrected data and number of singlets and triplets to the function
corrected_data_interpolate = correct_phases(datainterpolate,nsinglets,ntriplets)

In [9]:
# correct the rest of the data for which we have phasevectors found

corrected_data = correct_phases(data,nsinglets,ntriplets)
print(len(corrected_data), "points could be corected from the original data.")

# get relevant geometries

corrected_atoms = []
for i in range(len(atoms)):
    if "phases" not in data[i]:
        pass
    else:
        corrected_atoms.append(atoms[i])


2 points could be corected from the original data.


# Reading a trajectory file from SHARC dynamics

To read a trajectory file from SHARC, namely output.dat, make sure you already have converted the data (usually saved in netcdf format) to output.dat and output.xyz files. This can be data via the data-extractor file available with SHARC. Please refer to the SHARC tutorial for more information.

For reading such files, we need to specify the number of states and the threshold (0.5) for reliable overlaps. The rest will be read from the output.dat header.
Again, we need to give the path of the file.

In [10]:
from schnarc.utils import read_traj

traj_data = {}
traj_geoms = []

#we iterate over all files in 10 different trajectory folders,
#hence ntraj (number of trajectories) = 10
ntraj=10

idata = -1
for itraj in range((ntraj)):
    
    path = "./Trajectories/TRAJ_%05d/" %(itraj+1)
    datatraj_,atomstraj_ = read_traj(path,nsinglets,ntriplets,0.5)
    #put data into one large dictionary
    for i in range(len(datatraj_)):
        idata+=1
        traj_data[idata] = datatraj_[i]
        traj_geoms.append(ase.atoms.Atoms(atomstraj_[i].get_atomic_numbers(),atomstraj_[i].get_positions()/Bohr))

print(len(traj_data), "number of data points found.")

2410 number of data points found.


You can phase correct the data using the function from above if needed by parsing the traj_data data base instead of the previous data base.

# Making the data set 

The data set will be saved in ase format (https://wiki.fysik.dtu.dk/ase/ase/db/db.html).



In [11]:
from ase.db import connect

## Without phase correction

In [12]:
# we will only write the necessary properties into the data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newdata = {}
for i in range(len(data)):
    newdata[i]={}
    for key in keys:
        if key in data[i]:
            newdata[i][key] = data[i][key]
# define the name of the data set
#delete any DB that might be here
os.system("rm -f ../DBs/Fulvene.db")
dbname = "../DBs/Fulvene.db"
db = connect(dbname)
for i in range(len(data)):
    #the data are in the "data" dicitonary and the atoms saved as atoms objects in the "atoms"-array
    db.write(atoms[i],data=newdata[i])
    
#define metadata
metadata = {}
metadata["info"]="Write down any information you wish to remember later, e.g., reference method, if phasecorrected, etc."

# this information is required
metadata["n_singlets"]=nsinglets
metadata["n_triplets"]=ntriplets
db.metadata=metadata
print("We have ", len(db), " data points.")


We have  100  data points.


## Trajectory data

In [15]:
# we will only write the necessary properties into the traj_data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newtraj_data = {}
for i in range(len(traj_data)):
    newtraj_data[i]={}
    for key in keys:
        if key in traj_data[i]:
            newtraj_data[i][key] = traj_data[i][key]
# define the name of the traj_data set
#delete any DB that might be here
os.system("rm -f ../DBs/Fulvene_traj.db")
dbname = "../DBs/Fulvene_traj.db"
db = connect(dbname)
for i in range(len(traj_data)):
    #the traj_data are in the "traj_data" dicitonary and the atoms saved as atoms objects in the "atoms"-array
    db.write(traj_geoms[i],data=newtraj_data[i])
    
db.metadata=metadata
print("We have ", len(db), " data points.")


We have  2410  traj_data points.


## Phase corrected data

In [13]:

dbname = "../DBs/Fulvene_phasecorrected.db"
os.system("rm -f %s"%dbname)
db = connect(dbname)

# we will only write the necessary properties into the data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newdata_corrected = {}
for i in range(len(corrected_data)):
    newdata_corrected[i]={}
    for key in keys:
        if key in corrected_data[i]:
            newdata_corrected[i][key] = corrected_data[i][key]
# we will only write the necessary properties into the data file

newdata_corrected_interpolate = {}
for i in range(len(corrected_data_interpolate)):
    newdata_corrected_interpolate[i]={}
    for key in keys:
        if key in corrected_data_interpolate:
            newdata_corrected_interpolate[i][key] = corrected_data_interpolate[i][key]


for i in range(len(corrected_data_interpolate)):
    db.write(geoms[i],data=newdata_corrected_interpolate[i])
#now write the other corrected data
# note that the db will not be overwritten but data will be added with the write function

for i in range(len(corrected_data)):
    db.write(corrected_atoms[i],data=newdata_corrected[i])

# again define the metadata
metadata = {}
metadata["info"]="Write down any information you wish to remember later, e.g., reference method, if phasecorrected, etc."

# this information is required
metadata["n_singlets"]=nsinglets
metadata["n_triplets"]=ntriplets
db.metadata=metadata
print("We have ", len(db), " data points.")


We have  32  data points.


Congratulations! You have generated an initial training set for SchNarc and can now use it to train your models.

Note that if you want to expand the training set you can use the same functions, but make sure to not delete the data base in advance. See the next line to write the same data to the data base again and check the length of the training set.

In [18]:
# we will only write the necessary properties into the data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newdata = {}
for i in range(len(data)):
    newdata[i]={}
    for key in keys:
        if key in data[i]:
            newdata[i][key] = data[i][key]
# define the name of the data set
#delete any DB that might be here
dbname = "../DBs/Fulvene.db"
db = connect(dbname)
for i in range(len(data)):
    #the data are in the "data" dicitonary and the atoms saved as atoms objects in the "atoms"-array
    db.write(atoms[i],data=newdata[i])
    
#define metadata
metadata = {}
metadata["info"]="Write down any information you wish to remember later, e.g., reference method, if phasecorrected, etc."

# this information is required
metadata["n_singlets"]=nsinglets
metadata["n_triplets"]=ntriplets
db.metadata=metadata
print("We have ", len(db), " data points.")


We have  300  data points.
